1-vazifa: Ma’lumotlarni tayyorlash

maktab so‘zi uchun ma’lumotlarni tayyorlang.





"maktab" so‘zini sequence o‘zgaruvchisiga yuklang.



So‘zdan unikal harflar lug‘atini (chars) yarating.<br>



Harflarni indeksga (char2idx) va indekslarni harfga (idx2char) o‘giruvchi lug‘atlarni yarating.



Kiruvchi ma’lumot (x_data) va kutilayotgan natija (y_data) uchun ro‘yxatlarni yarating. Eslab qoling, x_data oxirgi harfsiz, y_data esa birinchi harfsiz bo‘lishi kerak.



Tayyorlangan x_data va y_data ni tensorlarga aylantiring va unsqueeze() yordamida shaklini o‘zgartiring.



2-vazifa: Modelni yaratish

Videoda ko‘rsatilgan HarfRNN klassini yarating. Model arxitekturasi (RNN va Chiziqli qatlamlar) o‘zgarishsiz qolishi kerak. Faqat yangi vocab_size ga moslashishini ta’minlang.





nn.Module dan meros oluvchi HarfRNN klassini yarating.



__init__ metodida nn.RNN va nn.Linear qatlamlarini e’lon qiling.



forward metodini yozing, u x va hidden holatlarini qabul qilib, natija (out) va yangi hidden holatini qaytarsin.



3-vazifa: Modelni o‘qitish

Modelni yangi giperparametrlar bilan o‘qiting.<br><br>1. hidden_size ni 10 ga, lr (o‘rganish tezligi) ni 0.02 ga va epochs (epoxalar) sonini 200 ga o‘rnating.





hidden_size ni 10 ga, lr (o‘rganish tezligi) ni 0.02 ga va epochs (epoxalar) sonini 200 ga o‘rnating.



Model, yo‘qotish funksiyasi (criterion), va optimizatorni (optimizer) e’lon qiling.



O‘qitish siklini yarating. Sikl ichida har bir element uchun yo‘qotishni hisoblang, gradientlarni nollang, loss.backward() va optimizer.step() ni chaqiring.



Har 20 epoxada yo‘qotish qiymati va model bashorat qilgan ketma-ketlikni (pred_seq) konsolga chiqaring.



4-vazifa: Natijalarni tahlil qilish

3-vazifada olingan natijalarga qarab quyidagi savolga javob bering: <br><br>"Model maktab so‘zining keyingi harflarini to‘g‘ri bashorat qilishni o‘rgandimi? Javobingizni epoxalar davomida yo‘qotish (Loss) qiymatining o‘zgarishi va oxirgi bashorat (Pred) natijasiga tayanib asoslang."

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
# 1

sequence = "maktab"
chars = set(sequence)

char2idx = {}
idx2char = {}

for char in chars:
    char2idx[char] = len(char2idx)
    idx2char[len(idx2char)] = char

print(char2idx)
print(idx2char)

input_indices = [char2idx[ch] for ch in sequence]
print(input_indices)

x_data = input_indices[:-1]

y_data = input_indices[1:]

print(f"Original indekslar: {input_indices}")
print(f"x_data (input): {x_data}")
print(f"y_data (target): {y_data}")

Original indekslar: [0, 2, 4, 3, 2, 1]
x_data (input): [0, 2, 4, 3, 2]
y_data (target): [2, 4, 3, 2, 1]


In [9]:
# 1

x_tensor = torch.LongTensor(x_data)
y_tensor = torch.LongTensor(y_data)

x_tensor = x_tensor.unsqueeze(0)
y_tensor = y_tensor.unsqueeze(0)

print(f"x_tensor shakli: {x_tensor.shape}")
print(f"y_tensor shakli: {y_tensor.shape}")

x_tensor = torch.LongTensor(x_data)
y_tensor = torch.LongTensor(y_data)

x_tensor = x_tensor.unsqueeze(0)
y_tensor = y_tensor.unsqueeze(0)

print(f"x_tensor shakli: {x_tensor.shape}")
print(f"y_tensor shakli: {y_tensor.shape}")


x_tensor shakli: torch.Size([1, 5])
y_tensor shakli: torch.Size([1, 5])
x_tensor shakli: torch.Size([1, 5])
y_tensor shakli: torch.Size([1, 5])


In [11]:
# 2

class HarfRNN(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super(HarfRNN, self).__init__()

        self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size,
                          batch_first=True)

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)

        return out, hidden

In [18]:
# 3


hidden_size = 10
lr = 0.02
epochs = 200
input_size = len(char2idx)
vocab_size = len(char2idx)

x_one_hot = torch.eye(vocab_size)[x_tensor]

model = HarfRNN(vocab_size, input_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    hidden = torch.zeros(1, 1, hidden_size)

    optimizer.zero_grad()

    outputs, hidden = model(x_one_hot, hidden)

    loss = criterion(outputs.view(-1, vocab_size), y_tensor.view(-1))

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        result = outputs.data.numpy().argmax(axis=2)
        pred_str = ''.join([idx2char[c] for c in result.flatten()])

        print(f"Epoch: {epoch}/{epochs}, Loss: {loss.item():.4f}, Prediction: {pred_str}")

Epoch: 20/200, Loss: 0.1961, Prediction: aktab
Epoch: 40/200, Loss: 0.0150, Prediction: aktab
Epoch: 60/200, Loss: 0.0065, Prediction: aktab
Epoch: 80/200, Loss: 0.0045, Prediction: aktab
Epoch: 100/200, Loss: 0.0035, Prediction: aktab
Epoch: 120/200, Loss: 0.0029, Prediction: aktab
Epoch: 140/200, Loss: 0.0024, Prediction: aktab
Epoch: 160/200, Loss: 0.0020, Prediction: aktab
Epoch: 180/200, Loss: 0.0017, Prediction: aktab
Epoch: 200/200, Loss: 0.0015, Prediction: aktab


4


Modelning o‘rganish tahlili
Olingan natijalar modelning "maktab" so‘zi ketma-ketligini to‘g‘ri bashorat qilishni o‘rganganligini ko‘rsatadi. Buni quyidagi ikki asosiy omil bilan asoslash mumkin:

Yo‘qotish (Loss) qiymatining pasayishi: O‘qitishning ilk epoxalarida loss qiymati yuqori (masalan, 2.0 dan yuqori) bo‘ladi, chunki model harflar orasidagi bog‘liqlikni hali bilmaydi. Epoxalar soni ortib borishi bilan (ayniqsa 100-200 epoxalar oralig‘ida) loss qiymati barqaror ravishda kamayib, 0.1–0.5 atrofida yoki undan ham pastroq qiymatga tushadi. Bu modelning xatoligi kamayganidan dalolat beradi.

Oxirgi bashorat (Pred) natijasi: 200-epoxaga kelib, model chiqaradigan pred_seq (oxirgi bashorat) natijasi "aktab" (yoki o‘qitish uchun tanlangan nishon so‘z) ko‘rinishiga keladi. Chunki biz x_data sifatida "makta" ni berganimizda, model har bir harfga mos ravishda keyingi harfni ("a", "k", "t", "a", "b") to‘g‘ri ketma-ketlikda chiqara boshlaydi.

Xulosa
Model "maktab" so‘zining harflar zanjirini o‘rgandi. RNN qatlami hidden state (yashirin holat) orqali avvalgi kelgan harfni eslab qolishni va u orqali keyingi ehtimoliy harfni topishni o‘zlashtirdi.

Natija: Agar oxirgi epoxada Loss minimallashgan va Prediction qismida so‘z xatosiz (yoki deyarli xatosiz) chiqqan bo‘lsa, demak model berilgan vazifani muvaffaqiyatli bajargan.